In [1]:
import pandas as pd
import numpy as np
import re
pd.options.mode.chained_assignment = None 

In [2]:
# Opening .xlsx file 

excelpath = input('Enter .xlsx file path: ')
df_excel = pd.read_excel(excelpath)

Enter .xlsx file path: /content/GAS12E01.xlsx


In [3]:
# Converting columns to lists for later

col_start = df_excel['Start,time'].tolist()
col_end = df_excel['End,time'].tolist()
col_pp = df_excel['PP'].tolist()
col_sp = df_excel['SP'].tolist()
col_mp = df_excel['MP'].tolist()

In [4]:
# Opening .srt file 

srtpath = input('Enter .srt file path: ')
with open(srtpath, 'r') as f:
  subs = f.read().splitlines()

Enter .srt file path: /content/Greys.Anatomy.S12E01.REPACK.HDTV.x264-KILLERS[eztv].it.srt


In [5]:
# Parsing .srt file

re_pattern = r'[0-9]{2}:[0-9]{2}:[0-9]{2},[0-9]{3} -->'
regex = re.compile(re_pattern)

start_times = list(filter(regex.search, subs))
start_times = [time.split(' ')[0] for time in start_times]
start_times_datetime = pd.to_datetime(start_times, format="%H:%M:%S,%f")

end_times = list(filter(regex.search,subs))
end_times = [time.split(' ')[2] for time in end_times]
end_times_datetime = pd.to_datetime(end_times, format="%H:%M:%S,%f")

subtitles = [[]]
for sub in subs:
    if re.match(re_pattern, sub):
        subtitles[-1].pop()
        subtitles.append([])
    else:
        subtitles[-1].append(sub)

subtitles = subtitles[1:]
subtitles = [' '.join(x) for x in subtitles]

In [6]:
# Computing the mean of each subtitle's timespan, which is used later
# to align the subs with the Excel segments

average_times = []

for x, y in zip(start_times_datetime, end_times_datetime):
  ts1 = x
  ts2 = y
  average_time = ts1+(ts2-ts1)/2
  average_times.append(average_time)

In [7]:
# By default, milliseconds are displayed in a six-digit format:

print(start_times_datetime[0])
print(end_times_datetime[0])

1900-01-01 00:00:00.893000
1900-01-01 00:00:02.567000


In [8]:
# This function removes the extra 0s (and the dates):

format_start = []
format_end = []

def format_time(x, y):
    s = x.strftime('%H:%M:%S.%f')
    s = s[:-3]                      # [:-3] cut extra digits / [:-7] cut milliseconds altogether
    y = y.strftime('%H:%M:%S.%f') 
    y = y[:-3]
    format_start.append(s)
    format_end.append(y)

for x, y in zip(start_times_datetime, end_times_datetime):
  format_time(x, y)

print(format_start[:5])
print(format_end[:5])

['00:00:00.893', '00:00:02.577', '00:00:04.464', '00:00:07.298', '00:00:08.465']
['00:00:02.567', '00:00:04.455', '00:00:07.288', '00:00:08.455', '00:00:10.692']


In [9]:
# Creating a DataFrame from the .srt data

df_subs = pd.DataFrame()
df_subs['Sub start'] = pd.Series(format_start)
df_subs['Sub end'] = pd.Series(format_end)
df_subs['Sub text'] = pd.Series(subtitles)
df_subs['Average'] = pd.Series(average_times).dt.time   

In [10]:
# Function that returns a df, which contains the subtitles that correspond
# to a segment in the Excel file, and appends it to a list

dfs_list = []

def filter_subs(start, end, pp, sp, mp):

    mask = (df_subs['Average'] > start) & (df_subs['Average'] <= end)   # Using the average that I calculated before
    mask_df = df_subs.loc[mask]

    if len(mask_df.index.values) == 0:

      empty_df_template = {
      'Start' : start,
      'End' : end,
      'Sub start' : ['NaN'],
      'Sub end' : ['NaN'],
      'Sub text' : ['NaN'],
      'PP' : pp,
      'SP' : sp,
      'MP' : mp,
      }

      empty_df = pd.DataFrame(empty_df_template)
      empty_df.replace('NaN',np.NaN)
      dfs_list.append(empty_df)

    else:

      mask_df['Start'] = start
      mask_df['End'] = end
      mask_df['PP'] = pp
      mask_df['SP'] = sp
      mask_df['MP'] = mp
      dfs_list.append(mask_df)

# Filtering all of the subtitles

for a, b, c, d, e in zip(col_start, col_end, col_pp, col_sp, col_mp):
  filter_subs(a, b, c, d, e)

In [11]:
# Combining the dfs and displaying the results in a nicer way

combined_data = pd.concat(dfs_list)
combined_data["Excel segment"] = combined_data["Start"].astype(str) + "-" + combined_data["End"].astype(str)
combined_data = combined_data.drop(['Start', 'End'], axis=1)
combined_data = combined_data[['Excel segment', 'Sub start', 'Sub end', 'PP', 'SP', 'MP', 'Sub text']]
combined_data = combined_data .rename(columns={'Sub start':'Subtitle start', 'Sub end':'Subtitle end', 'Sub text':'Subtitle text'})
combined_data.reset_index(drop=True)

,Excel segment,Subtitle start,Subtitle end,PP,SP,MP,Subtitle text
0,00:00:00-00:00:07,00:00:00.893,00:00:02.567,NaN,NaN,NaN,<i>Ognuno di voi e' venuto qui oggi carico di ...
1,00:00:00-00:00:07,00:00:02.577,00:00:04.455,NaN,NaN,NaN,<i>Ho cinque regole. Memorizzatele.</i>
2,00:00:00-00:00:07,00:00:04.464,00:00:07.288,NaN,NaN,NaN,<i>Qualcuno sa dirmi le cause piu' comuni dell...
3,00:00:07-00:00:28,00:00:07.298,00:00:08.455,0.0,6.0,0.0,<i>Magari penserete...</i>
4,00:00:07-00:00:28,00:00:08.465,00:00:10.692,0.0,6.0,0.0,"<i>Regola numero 5. Quando mi muovo io, vi muo..."
...,...,...,...,...,...,...,...
845,00:40:53-00:40:54,00:40:51.934,00:40:54.166,0.0,6.0,0.0,<i>Posizionatevi sotto il processo xifoideo......
846,00:40:54-00:40:55,NaN,NaN,6.0,0.0,0.0,NaN
847,00:40:55-00:40:58,00:40:55.251,00:40:56.902,0.0,6.0,0.0,<i>Premete con fermezza...</i>
848,00:40:58-00:41:00,00:40:59.039,00:41:00.489,6.0,0.0,0.0,<i>E senza alcun rimpianto...</i>
